In [1]:
library(dplyr)
library(SeuratDisk)
library(Seurat)
#library(SeuratData)
library(cowplot)
library(Azimuth)
library(dplyr)
library(cTPnet)
library(reticulate)
use_condaenv(condaenv = "seurat", required = TRUE)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Attaching SeuratObject

Attaching shinyBS

Loading required package: reticulate



In [2]:
#names = c("wang_2020", "butler_2019", "witkowski_2020", "fournie_2020")
names = c("kotliarov")
for (j in 1:4){
    name = names[j]
    rna <- LoadH5Seurat(paste("seuratfiles/", name, ".h5seurat", sep = ""))
    adt <- LoadH5Seurat(paste("seuratfiles/",name, "-adt.h5seurat", sep = ""))
    rna[["ADT"]] <- CreateAssayObject(counts = adt@assays$ADT@counts)

    seeds = c(42, 43, 57, 12, 123)
    azi = readRDS(paste("seuratfiles/", name, '_azimuth_impADT.Rds',sep = ""))
    rna[["Azimuth"]] <- CreateAssayObject(data = azi@data)
    rna = cTPnet(rna,data_type= "Seurat3", "cTPnet_weight_24")
    for (i in 1:5){
        set.seed(seeds[i]) # 42, 43, 57, 12, 123
        sampled.cells <- sample(Cells(rna), size = round(length(Cells(rna))*0.8), replace = F)
        rna1 <- subset(rna, cells = sampled.cells) 
        SaveH5Seurat(rna1, paste("seuratfiles/",name, "-complete-run",as.character(i),".h5seurat",sep = ""))
        }
    }

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding scale.data for ADT

Adding feature-level metadata for ADT

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



Start data preprocessing...
Start imputation. Running python ...
Postprocess...
Done!


ERROR: Error: H5Seurat file at seuratfiles/wang_2020-complete-run1.h5seurat already exists


In [2]:
rna <- LoadH5Seurat(paste("seuratfiles/", "kotliarov_2020", ".h5seurat", sep = ""))
adt <- LoadH5Seurat(paste("seuratfiles/", "kotliarov_2020", "-adt.h5seurat", sep = ""))

Validating h5Seurat file

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Initializing RNA with data



ERROR: Error in sparseMatrix(i = x[["indices"]][] + 1, p = x[["indptr"]][], x = x[["data"]][], : all(dims >= dims.min) is not TRUE


# Extracting Scaled Data

In [49]:
name = "butler_2019"
df <- LoadH5Seurat(paste("seuratfiles/", name, "-complete.h5seurat", sep = ""))

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding miscellaneous information for ADT

Initializing Azimuth with data

Adding miscellaneous information for Azimuth

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding miscellaneous information for RNA

Initializing cTPnet with data

Adding miscellaneous information for cTPnet

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [50]:
DefaultAssay(df) <- 'RNA'
df <- NormalizeData(df) %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA()

DefaultAssay(df) <- 'ADT' 
VariableFeatures(df) <- rownames(df[["ADT"]])
df <- NormalizeData(df, normalization.method = 'CLR', margin = 2) %>% 
  ScaleData() %>% RunPCA(reduction.name = 'apca')

DefaultAssay(df) <- 'Azimuth' 
VariableFeatures(df) <- rownames(df[["Azimuth"]])
df <- ScaleData(df) %>% RunPCA(reduction.name = "azipca")

DefaultAssay(df) <- 'cTPnet'
VariableFeatures(df) <- rownames(df[["cTPnet"]])
df <- ScaleData(df) %>% RunPCA(reduction.name = 'ctppca')

Centering and scaling data matrix

PC_ 1 
Positive:  TRBC1, LAT, CCL5, CD8B, S100A12, S100A8, S100A9, IGKC, KLRB1, TYROBP 
	   VCAN, GZMA, FCN1, CD14, MS4A1, SERPINA1, RBP7, GNLY, S100B, CST7 
	   KLRD1, RP11-291B21.2, NKG7, CCL4, IGLC2, AC092580.4, IGLC3, PRF1, DUSP2, CYP1B1 
Negative:  KIAA0101, KLF1, TYMS, FAM178B, KCNH2, CNRIP1, APOC1, CENPU, GATA1, BIRC5 
	   EPCAM, RP11-620J15.3, CENPF, TFR2, STMN1, CKS2, TUBA1B, HIST1H4C, HMGA1, CDT1 
	   CA1, TK1, AHSP, TOP2A, GFI1B, TUBB, NME4, MKI67, TMEM56, SMIM1 
PC_ 2 
Positive:  LYZ, FCN1, CST3, TYROBP, S100A9, LST1, S100A8, VCAN, LGALS1, AIF1 
	   S100A12, CFD, SERPINA1, FOS, FCER1G, MS4A6A, S100A6, CD14, LGALS2, FTH1 
	   GAPDH, CD36, ANXA2, CPVL, RBP7, HLA-DRA, H3F3A, LINC01272, PLBD1, RETN 
Negative:  RPL3, RPS3, RPS18, RPS4X, RPS5, RPS12, RPSA, RPS2, RPL14, RPS23 
	   EEF1B2, RPL4, LDHB, NPM1, RPS17, RPLP0, TRBC1, LAT, RPL7A, GYPC 
	   HSPA8, CD8B, KLRB1, HNRNPA1, CCL5, RPL37A, PEBP1, MYC, CD79A, SOD1 
PC_ 3 
Positive:  AHSP, GYPA, A

In [51]:
DefaultAssay(df) <- 'RNA'
df@active.ident <- df$cell_type_l2
markers <- FindAllMarkers(df, verbose = TRUE)

Calculating cluster CD16+ NK

Calculating cluster CD16- NK

Calculating cluster Treg

Calculating cluster classical monocyte

Calculating cluster intermediate monocyte

Calculating cluster memory B

Calculating cluster memory CD4

Calculating cluster memory CD8

Calculating cluster myeloid DC

Calculating cluster naive B

Calculating cluster naive CD4

Calculating cluster naive CD8

Calculating cluster non-classical CD16+ monocyte

Calculating cluster plasmacytoid DC

Calculating cluster undefined



In [52]:
markers

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
GNLY,0,4.893798,0.995,0.130,0,CD16+ NK,GNLY
NKG7,0,4.090386,0.996,0.154,0,CD16+ NK,NKG7
GZMB,0,3.689890,0.862,0.045,0,CD16+ NK,GZMB
KLRF1,0,3.190195,0.793,0.029,0,CD16+ NK,KLRF1
CST7,0,3.110669,0.900,0.102,0,CD16+ NK,CST7
KLRD1,0,3.070496,0.806,0.050,0,CD16+ NK,KLRD1
PRF1,0,3.064088,0.742,0.045,0,CD16+ NK,PRF1
SPON2,0,3.035038,0.638,0.027,0,CD16+ NK,SPON2
FGFBP2,0,2.981255,0.631,0.023,0,CD16+ NK,FGFBP2


In [53]:
length(intersect(VariableFeatures(df), markers$gene))

[1] 842

In [54]:
length(union(markers$gene, c(1,2)))

[1] 3704

In [31]:
write.csv(df@assays$RNA@scale.data, paste("csvfiles/", name, "_rna_scaledata.csv", sep = ""))
write.csv(df@assays$ADT@scale.data, paste("csvfiles/", name, "_adt_scaledata.csv", sep = ""))
write.csv(df@assays$Azimuth@scale.data, paste("csvfiles/", name, "_azi_scaledata.csv", sep = ""))
write.csv(df@assays$cTPnet@scale.data, paste("csvfiles/", name, "_ctp_scaledata.csv", sep = ""))

In [32]:
write.csv(df@reductions$pca@cell.embeddings, paste("csvfiles/", name, "_rna_pca.csv", sep = ""))
write.csv(df@reductions$apca@cell.embeddings, paste("csvfiles/", name, "_adt_pca.csv", sep = ""))
write.csv(df@reductions$azipca@cell.embeddings, paste("csvfiles/", name, "_azi_pca.csv", sep = ""))
write.csv(df@reductions$ctppca@cell.embeddings, paste("csvfiles/", name, "_ctp_pca.csv", sep = ""))

In [17]:
write.csv(df$cell_type_l1, paste("csvfiles/", name, "_cell_type_l1.csv", sep =""))
write.csv(df$cell_type_l2, paste("csvfiles/", name, "_cell_type_l2.csv", sep =""))
write.csv(df$cell_score_l1, paste("csvfiles/", name, "_cell_score_l1.csv", sep =""))
write.csv(df$cell_score_l2, paste("csvfiles/", name, "_cell_score_l2.csv", sep =""))

ACTCTCGTCAGCGTCG AGCGTCGCACAGCCAC TCGTGGGGTTTCCAAG ATCGTGAAGTCGAATA 
       undefined        undefined        undefined        undefined 
CAGAGCCTCGCTCCTA CGAGTGCGTCGTTGCG GTCGTAACATGTCGTA TCGTGGGAGAATCGCG 
       undefined        undefined        undefined        undefined 
TCGTGGGGTGCTCGTG AATCGTGCAGACCTGC AGCGTCGGTACTGTTG CGTGTCTCAACTCCAA 
       undefined        undefined        undefined        undefined 
TTCACGCGTCGTTATG AGCGTCGCAATACGAA CCGGGTAGTAGATGTA TACGTCCAGCGTCGAA 
       undefined        undefined        undefined        undefined 
TCCATGCGTCGTTCAA TCCTGCAGTGATATAG TCGTGGGTCAGGGATG AGCGTATGTGTAGGAC 
       undefined        undefined        undefined        undefined 
CTCCAACGTCACTACA GAGACTTTCGCGTCGA GGGTCACGTCGTGGAA GGGTGTCGTGTACAGG 
           CD8 T        undefined        undefined        undefined 
GTCATTTTCGCGTCGA TCGGGTGGTGGACTGA AGCCAATGTGCGTCGT ACCGTTCGTCGTGCCA 
       undefined        undefined        undefined        undefined 
GTCATGAAGGCCACTC TCATACTTCTCCAAGA TCCGTGTCACACAGAG TCGCTTGTCGGAAACG 
           CD4 T            CD8 T            CD4 T        monocytes 
TCGTCCAGTAGTACGG CAAGCTAGTTTACGAC CATCGTCGTCTGTGTA CGTGTCTGTGGCGCTT 
       undefined            CD4 T        monocytes        undefined 
GTCGTTCGTTAGCGGA GTGCTTCGTGTTCAGT TCGCACTGTACGCGTC TTACGCCGTGTTGATC 
           CD4 T        monocytes        undefined        undefined 
ACGATGTTCAGCGTCG CTGTGAAGTCGTGGTC GCCGTGACATTCGATG GTCATTTTCGATGCAT 
       undefined            CD4 T            CD4 T            CD4 T 
GTCGCGACACCCTGAG GTCGTTCTCGTTCATT TAAGCCAAGTTAGTGA TCATGGAGTGGCTTGC 
       monocytes            CD4 T        monocytes            CD8 T 
TCCGTGTGTCAGTCGC TCGCACTTCCGCCTAT TCGTAGAAGTACCATC TCGTGGGTCTATCGTT 
           CD4 T            CD4 T            CD4 T        undefined 
TTTCATGTCCTATGGA ATTGGGTAGGTACCTT CACAGGCGTCGTTGGC CGTGCTTAGTCAGGGT 
       undefined            CD8 T        undefined        monocytes 
CGTGTCTCAGTTGTCA GCCCAGAAGTGCGTCC GGAGGTAAGAGCATCG GGCGTCAAGGCACTAG 
       undefined            CD4 T        undefined            CD4 T 
TCGGGTGGTTTGGAAA TTTCAGTTCAGCGTCG AGAAGCGTCTCTGCTG ATTGTTCTCGTTCGCT 
           CD4 T        undefined        monocytes            CD8 T 
CACGAATAGGCTGGAT CAGTGCGTCCAGTGTA CATCGGGAGCGCCTTG CTAACCCAGGGAGGAC 
       monocytes        monocytes        undefined            CD4 T 
CTACAGAAGACGTCGA CTGAGGCGTCGAGATG GCGTGCACAAGTGTCT GTCGCGATCTGCACCT 
       monocytes        undefined            CD4 T        undefined 
GTCGTAAGTATCAAGA TCGGATAGTGAGCTCC TCGTAGAAGCTGAAAT AAGCGTTTCGTGGACC 
       undefined        monocytes            CD4 T            CD4 T 
CGACAGCGTCAAGGCA CGTGCTTAGGGCATGT CGTGTCTAGGTTGCCC CTATCCGTCGCGTGCA 
           CD4 T        monocytes        undefined            CD4 T 
CTGATCCCATGGCACC GTCGAATAGTGAGGTC GTCGAATGTCAGCGTC TAGAGTCGTCGATGCC 
       undefined        undefined        monocytes        monocytes 
TCACTATGTTCTTCAT TCCTCGAGTAGGTGCA TCGACGGCATGATGCT TCGTCCAGTGACGTCC 
       undefined        undefined            CD8 T                B 
TGCTGAAAGGCTTAGG TGTCCTGCAGGCTTGC TTGCGTCCAGGCCCTA AATCGACAGTTGCGCC 
               B            CD4 T            CD4 T        undefined 
ATGCGATGTTGGTAGG ATTCAGGGTGACTCTA ATTCCATAGCGTCTCG CAGTGCGTCATTCGGA 
           CD4 T            CD8 T            CD4 T            CD4 T 
CCGTGAGCACAAGTGG CGTAATGAGCCGGATA CGTGCTTGTCAGCGTC CTGTCGTGTTCTCTCG 
           CD4 T            CD8 T            CD4 T        undefined 
GTCGTAAAGGCCCACT GTCGTAATCCTGCTAC TCGGGACAGCACTCCG TCGTAGATCATCTACT 
       undefined        undefined            CD4 T            CD4 T 
TGTAGACGTCGCAGTC AGCGCCAGTGCCAAGA AGGACTTCAGACGCTC CCGTGAGTCCACAGCG 
       monocytes        monocytes        monocytes            CD4 T 
CGAATTGTCTGTTGGA CGCGTGATCGCTAGCG CTCCACACACGTGTGC GATGATCCAGAGCGTA 
           CD4 T            CD8 T            CD4 T            CD4 T 
GCGATCGTCGCACGAC GGCGTCAGTTCAGGTT TCGAAGTCACCGTACG TCGACGGAGAGAACCC 
       undefined            CD4 T        monocytes            CD4 T